import libraries

In [3]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
import spacy      #for grammers
from tqdm import tqdm

load spacy 

In [9]:
#!pip install contractions textblob emoji spacy
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 409.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 433.4 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 434.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 447.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 428.6 kB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


download data

In [4]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/melika_yazdanpanah/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/melika_yazdanpanah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
class TextPreprocessor:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        self.stemmer = PorterStemmer() #stemming
        self.lemmatizer = WordNetLemmatizer() #lemmatization
        self.stop_words = set (stopwords.words('english'))
        self.tokenizer = TweetTokenizer()

    def preprocess(self, textdata):
        processedText = []

        for tweet in tqdm(textdata, desc="Processing tweets", unit="tweet"):
            tweet = re.sub(r"\'ll", " will", tweet)
            tweet = re.sub(r"\'re", " are", tweet)
            tweet = re.sub(r"\'ve", " have", tweet)
            tweet = re.sub(r"\'d", " would", tweet)
            tweet = re.sub(r"\'s", " is", tweet)
            tweet = re.sub(r"\'m", " am", tweet)
            
            
            tweet = tweet.lower()
            
            tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE) #urls
            tweet = re.sub(r"@\w+", '', tweet) #users
            tweet = re.sub(f"[{string.punctuation}]", '', tweet) # punctuation
            tweet = re.sub(r'\d+', '', tweet) #numbers
            
            #tokenize
            tokens = self.tokenizer.tokenize(tweet)
            
            tokens = [word for word in tokens if word not in self.stop_words] #stop words
            
            #stemming and lemmatization
            tokens = [self.stemmer.stem(self.lemmatizer.lemmatize(word)) for word in tokens]
            doc = self.nlp(' '.join(tokens)) #name entities
            tweet = ' '.join([token.text if not token.ent_type_ else "ENTITY" for token in doc])
            
            
            processedText.append(tweet)

        return processedText


**TEST**

In [11]:
# Make data directory if it doesn't exist
"""!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/sentiment140-subset.csv.zip -P data
!unzip -n -d data data/sentiment140-subset.csv.zip"""

--2024-08-16 22:01:30--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/sentiment140-subset.csv.zip
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17927149 (17M) [application/zip]
Saving to: ‘data/sentiment140-subset.csv.zip’

sentiment140-subset 100%[===================>]  17.10M   410KB/s    in 42s     

2024-08-16 22:02:14 (413 KB/s) - ‘data/sentiment140-subset.csv.zip’ saved [17927149/17927149]

/bin/bash: line 1: unzip: command not found


In [18]:

"""df = pd.read_csv("/home/melika_yazdanpanah/amin/I-m-not-Sentimental/Preprocessing/data/sentiment140-subset.csv.zip", encoding="ISO-8859-1")
df_sample = df.head(10)
text_preprocessor = TextPreprocessor()
df_sample['text'] = text_preprocessor.preprocess(df_sample['text'].values)
print(df_sample.head())"""

Processing tweets: 100%|██████████| 10/10 [00:00<00:00, 222.88tweet/s]

   polarity                                               text
0         0                                        never tweet
1         0                             sick ENTITY code couch
2         1  thx answer ENTITY afraid gon na crash twitter ...
3         1           ENTITY fit say lost pound sinc last time
4         0                            thing do nt realli life



/tmp/ipykernel_1430/3820438114.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['text'] = text_preprocessor.preprocess(df_sample['text'].values)
